# Import Library

In [120]:
# import requests
# from bs4 import BeautifulSoup
# import csv
# import time
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from langdetect import detect, LangDetectException
from textblob import TextBlob
from transformers import pipeline
# from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
# from sklearn.preprocessing import LabelEncoder
# from keras.models import Sequential
# from keras.layers import Embedding, GRU, Dense
from sklearn.feature_extraction.text import TfidfVectorizer
# from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder










# Data Load

In [ ]:



df = pd.read_csv("letterboxd_reviews_clean.csv")


df = df.drop(columns=["Username"])


df['Movie Name'] = df['Movie Name'].apply(lambda name: (
    'Agak Laen' if  'Agak Laen' in name else
    'Vengeance Is Mine, All Others Pay Cash' if 'Vengeance' in name else name
))

df.sample(20)


,Movie Name,Review,Rating
3189,"Vengeance Is Mine, All Others Pay Cash","Anjingg, KerenBercerita tentang seorang pemuda...",5.0
3040,"Vengeance Is Mine, All Others Pay Cash",3 stars for the production design and costume ...,4.0
476,Agak Laen,panjaaangggg bet dan part komedi yg pecah buat...,3.0
3166,"Vengeance Is Mine, All Others Pay Cash",pros :- brutal & expertly crafted martial arts...,NaN
2595,"Vengeance Is Mine, All Others Pay Cash","I was expecting more action and comedy, tbh.",2.5
2933,"Vengeance Is Mine, All Others Pay Cash",Kill Bill vibes.,NaN
826,Agak Laen,alamak setannya batak,4.0
2236,"Vengeance Is Mine, All Others Pay Cash",satu dari sedikit film adaptasi novel yang ber...,4.0
2952,"Vengeance Is Mine, All Others Pay Cash","""Aku menceintaimu iteung, tetapi aku tak bisa ...",5.0
3408,"Vengeance Is Mine, All Others Pay Cash","Seperti Dendam, Rindu Harus Dibayar Tuntasfill...",NaN


# Data Cleaning

In [4]:


df.duplicated().sum()
df.drop_duplicates()



df_cleaned = df.dropna()

df_cleaned = df_cleaned[df_cleaned['Review'].str.strip().astype(bool)]

df_cleaned.info()
df_cleaned.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 3486 entries, 0 to 3599
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Movie Name  3486 non-null   object 
 1   Review      3486 non-null   object 
 2   Rating      3486 non-null   float64
dtypes: float64(1), object(2)
memory usage: 108.9+ KB


,Rating
count,3486.000000
mean,3.788583
std,0.840370
min,0.500000
25%,3.500000
50%,4.000000
75%,4.500000
max,5.000000


# Data Pre-Processing
- Konversi Lowercase
- Penghilangan karakter (i.e $,%,^,etc)
- Penghilangan Emoji
- Lemmatizer
- Stemmer
- Stopwords
- Tokenisasi
- Kembali ke bentuk kalimat

Semua dilakukan untuk kedua bahasa Inggris dan Indonesia

In [ ]:

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:



def remove_emoji(text):
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  
                        u"\U0001F300-\U0001F5FF"  
                        u"\U0001F680-\U0001F6FF"  
                        u"\U0001F1E0-\U0001F1FF"  
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

lemmatizer = WordNetLemmatizer()
stop_words_en = set(stopwords.words('english'))


factory = StemmerFactory()
indo_stemmer = factory.create_stemmer()

try:
    stop_words_id = set(stopwords.words('indonesian'))  
except:
    stop_words_id = set()  


all_stopwords = stop_words_en.union(stop_words_id)


def preprocess_text(text):
    text = text.lower()
    text = remove_emoji(text)
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)

    processed_tokens = []
    for word in tokens:
        if word in all_stopwords:
            continue
        try:
            
            lemma = lemmatizer.lemmatize(word)
            processed_tokens.append(lemma)
        except:
            
            stem = indo_stemmer.stem(word)
            processed_tokens.append(stem)

    return ' '.join(processed_tokens)


df_cleaned['Processed Review'] = df_cleaned['Review'].apply(preprocess_text)



In [ ]:
print(df_cleaned.head(10))





   Movie Name                                             Review  Rating  \
0   Agak Laen  Nggak ada yang ngasih tau kalau ini film prakt...     3.5   
1   Agak Laen  THIS IS SO FRESHHHHHH FROM COMEDY HORROR INDON...     5.0   
2   Agak Laen  Menurut saya film ini sangat amat lucu, setiap...     5.0   
4   Agak Laen  dari awal release udah rewatch sampe 100 kali ...     3.5   
5   Agak Laen                                      Agak Laen bah     5.0   
6   Agak Laen  It was a fun and fresh thing to bring haunted ...     4.0   
7   Agak Laen                             ketawa habis habisannn     4.0   
8   Agak Laen  mmm lucu sih tp ga selucu itu jg soalnya jokes...     2.0   
9   Agak Laen  Alur FRESHHH dan gua rasa ini REAL CRIMEEE BRO...     3.0   
10  Agak Laen                          surprisingly entertaining     4.0   

                                     Processed Review  
0             nggak ngasih tau film praktik pesugihan  
1   freshhhhhh comedy horror indonesia udh ga nont.

# Data Labelisasi
- Pada tahap ini, saya menerapkan deteksi bahasa, karena keseluruhan ulasan menggunakan bahasa Inggris dan bahasa Indonesia
- Bahasa Inggris menggunakan TextBlob untuk mendeteksi Sentimen
- Bahasa Indonesia menggunakana indobert-base-p1 untuk mendeteksi Sentimen

In [8]:

indonesian_sentiment_model = pipeline("sentiment-analysis", model="indobenchmark/indobert-base-p1", framework="pt")


def detect_language(text):
    try:
        if not text or not text.strip():
            return 'id' 
        lang = detect(text)
        return lang if lang in ['id', 'en'] else 'id'
    except LangDetectException:
        return 'id'
    
def get_sentiment(text, lang):
    if lang == "en":

        blob = TextBlob(text)
        polarity = blob.sentiment.polarity
        if polarity > 0:
            return "positive"
        else:
            return "negative"
    elif lang == "id":

        result = indonesian_sentiment_model(text)
        if result[0]['label'] == 'LABEL_1':
            return "positive"
        else:
            return "negative"
    else:

        return "neutral"
    



def apply_sentiment(df):

    df['Language'] = df['Review'].apply(detect_language)
    

    df['Sentiment'] = df.apply(lambda row: get_sentiment(row['Review'], row['Language']), axis=1)
    
    return df




df_sentiment = apply_sentiment(df_cleaned)


print(df_sentiment[['Review', 'Language', 'Sentiment']])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


                                                 Review Language Sentiment
0     Nggak ada yang ngasih tau kalau ini film prakt...       id  positive
1     THIS IS SO FRESHHHHHH FROM COMEDY HORROR INDON...       id  positive
2     Menurut saya film ini sangat amat lucu, setiap...       id  positive
4     dari awal release udah rewatch sampe 100 kali ...       id  positive
5                                         Agak Laen bah       id  positive
...                                                 ...      ...       ...
3594  Ratu Felisha is so horrifying in a good way.PS...       en  positive
3595  Asik, gendeng, dan liar betul nih perjalanan a...       id  positive
3596  Baru ini nonton film Indonesia yang banyak ken...       id  positive
3598  Sebenernya isu2 yg bersembunyi di film ini tuh...       id  positive
3599  Film yang cukup liar,brutal dan nyentrik mungk...       id  positive

[3486 rows x 3 columns]


In [9]:
df_sentiment.to_csv('df_sentiment.csv')


In [10]:
df_sentiment.groupby(['Language', 'Sentiment']).agg('count')



Movie Name  Review  Rating  Processed Review
Language Sentiment                                              
en       negative          433     433     433               433
         positive          868     868     868               868
id       negative           77      77      77                77
         positive         2108    2108    2108              2108

In [95]:

X = df_sentiment['Processed Review']
y = df_sentiment['Sentiment']

# Feature Extraction
- Menggunakan TF-IDF

In [96]:


tfidf = TfidfVectorizer(
    min_df=3,
    max_df=0.8,
    ngram_range=(1, 2),
    sublinear_tf=True,
    norm='l2'
)



X_tfidf = tfidf.fit_transform(X)



In [97]:

features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

features_df

,10,10 minute,1010,114,15,16,16 mm,16mm,16mm film,18,...,yg gw,yg kek,yg lucu,yg ngaceng,yg nonton,you,young,youtube,yuni,zaman
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Split

In [98]:

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=63)

# Data Modelling
Secara berturut-turut berikut 6 cara kombinasi percobaan modelling:


    Pelatihan: Logistic Regression,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20
    Pelatihan: SVM,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20
    Pelatihan: RF,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20
    Pelatihan: RF,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 70/30
    Pelatihan: Decision Tree,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20
    Pelatihan: Deep Learning Dense Layer with ReLu Activation, Ekstraksi Fitur: TF-IDF, Pembagian Data: 80/20      

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the logistic regression model
logreg = LogisticRegression(max_iter=200)

# Fit the model
logreg.fit(X_train, y_train)

# Make predictions
y_pred_train_lr = logreg.predict(X_train)
y_pred_test_lr = logreg.predict(X_test)

# Calculate accuracy
accuracy_train_lr = accuracy_score(y_train, y_pred_train_lr)
accuracy_test_lr = accuracy_score(y_test, y_pred_test_lr)

# Print results
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.8597560975609756
Logistic Regression - accuracy_test: 0.8510028653295129


In [100]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM model (with a linear kernel for simplicity)
svm = SVC(kernel='linear')

# Fit the model
# svm.fit(X_train, y_train)
svm.fit(X_train.toarray(), y_train)

# Make predictions
y_pred_train_svm = svm.predict(X_train.toarray())
y_pred_test_svm = svm.predict(X_test.toarray())

# Calculate accuracy
accuracy_train_svm = accuracy_score(y_train, y_pred_train_svm)
accuracy_test_svm = accuracy_score(y_test, y_pred_test_svm)

# Print results
print('SVM - accuracy_train:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)


SVM - accuracy_train: 0.9092539454806313
SVM - accuracy_test: 0.8638968481375359


In [101]:
from sklearn.ensemble import RandomForestClassifier


random_forest = RandomForestClassifier()


random_forest.fit(X_train.toarray(), y_train)


y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())


accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9913916786226685
Random Forest - accuracy_test: 0.8553008595988538


In [102]:
from sklearn.ensemble import RandomForestClassifier

X2_train, X2_test, y2_train, y2_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=63)


random_forest = RandomForestClassifier()


random_forest.fit(X2_train.toarray(), y2_train)

y2_pred_train_rf = random_forest.predict(X2_train.toarray())
y2_pred_test_rf = random_forest.predict(X2_test.toarray())


accuracy_train_rf = accuracy_score(y2_pred_train_rf, y2_train)
accuracy_test_rf = accuracy_score(y2_pred_test_rf, y2_test)


print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.990983606557377
Random Forest - accuracy_test: 0.858508604206501


In [103]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()


decision_tree.fit(X_train.toarray(), y_train)


y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())


accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)


print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.9913916786226685
Decision Tree - accuracy_test: 0.8237822349570201


In [ ]:
X2 = X_tfidf.toarray()
y2 = df_sentiment['Sentiment']


le = LabelEncoder()
y2_int = le.fit_transform(y2)
y2_ohe = to_categorical(y2_int, num_classes=2)

scaler = preprocessing.MinMaxScaler()
X2_scaled = scaler.fit_transform(X2)

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2_scaled, y2_ohe, test_size=0.2, random_state=42
)


early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


model = Sequential([
    Dense(128, input_shape=(X2_train.shape[1],), activation='relu',
    kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(34, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(15, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dense(2, activation='softmax')
])


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


history = model.fit(
    X2_train, y2_train,
    epochs=50,
    batch_size=32,
    validation_data=(X2_test, y2_test),
    callbacks=[early_stop],
    verbose=1
)

loss_train, acc_train = model.evaluate(X2_train, y2_train, verbose=0)
loss_test, acc_test = model.evaluate(X2_test, y2_test, verbose=0)

print(f"Train Accuracy: {acc_train:.4f}")
print(f"Test Accuracy : {acc_test:.4f}")


Epoch 1/50
88/88 [==============================] - 5s 19ms/step - loss: 1.0389 - accuracy: 0.5907 - val_loss: 0.7924 - val_accuracy: 0.8309
Epoch 2/50
88/88 [==============================] - 1s 14ms/step - loss: 0.7296 - accuracy: 0.7981 - val_loss: 0.7234 - val_accuracy: 0.8309
Epoch 3/50
88/88 [==============================] - 1s 15ms/step - loss: 0.6509 - accuracy: 0.8494 - val_loss: 0.7122 - val_accuracy: 0.8309
Epoch 4/50
88/88 [==============================] - 1s 14ms/step - loss: 0.5935 - accuracy: 0.8809 - val_loss: 0.7176 - val_accuracy: 0.8309
Epoch 5/50
88/88 [==============================] - 1s 16ms/step - loss: 0.5278 - accuracy: 0.9049 - val_loss: 0.7398 - val_accuracy: 0.8324
Epoch 6/50
88/88 [==============================] - 1s 16ms/step - loss: 0.4816 - accuracy: 0.9232 - val_loss: 0.7539 - val_accuracy: 0.8281
Train Accuracy: 0.8594
Test Accuracy : 0.8309


# Inference Test
-Menggunakan algoritma SVM, karena algoritma tersebut mencapai akurasi test yang paling tinggi.

In [ ]:
kalimat_baru = input("Masukkan kalimat baru: ")


testxax = preprocess_text(kalimat_baru)


X_kalimat_baru = tfidf.transform([testxax])


prediksi_sentimen = svm.predict(X_kalimat_baru.toarray())


if prediksi_sentimen[0] == 'positive':
    print(prediksi_sentimen)
    print(f"Input Kalimat Baru : {kalimat_baru}")
    print("Sentimen kalimat baru adalah POSITIF.")
else:
    print(prediksi_sentimen)
    print(f"Input Kalimat Baru : {kalimat_baru}")
    print("Sentimen kalimat baru adalah NEGATIF.")


['positive']
Input Kalimat Baru : filmnya bagus seru bgt
Sentimen kalimat baru adalah POSITIF.
